In [1]:
from PIL import Image
import os

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

#error message 제거
import os; os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'


In [2]:
image_dir = "/home/jwjang/project_tf28/image/"

IMAGE_SIZE = 224
BATCH_SIZE = 64

#pre_processing
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    validation_split = 0.1
)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    validation_split = 0.1
)

train_datagen = train_datagen.flow_from_directory(
    image_dir,
    target_size = (IMAGE_SIZE, IMAGE_SIZE),
    batch_size = BATCH_SIZE,
    subset = 'training'
)

test_datagen = test_datagen.flow_from_directory(
    image_dir,
    target_size = (IMAGE_SIZE, IMAGE_SIZE),
    batch_size = BATCH_SIZE,
    subset = 'validation'
)

Found 531 images belonging to 2 classes.
Found 58 images belonging to 2 classes.


tf.keras.preprocessing.image_dataset_from_directory(
    directory, #데이터가 위치한 디렉토리
    labels="inferred", #디렉토리에서 찾은 이미지 파일 수와 동일한 크기의 정수 목록/튜플, 레이블은 이미지 파일 경로의 영숫자 순서에 따라 정렬되어야함
    label_mode="int", #레이블이 정수로 인코딩됨을 의미
    class_names=None, #if)labels=inferred인 경우 클래스이름의 명시적 목록
    color_mode="rgb", #"grayscale", "rgb", "rgba" 중 하나, 이미지의 분류 채널 결정
    batch_size=32, #데이터 배치의 크기, 기본값:32
    image_size=(256, 256), #디스크에서 읽은 후 이미지 크기를 조정할 크기, 기본값:(256,256)
    shuffle=True, #데이터를 섞을지 여부
    seed=None, #셔플 및 변환을 위한 선택적 임시 시드
    validation_split=None, #0과 1사이의 선택적 부동 소수점, 유효성 검사를 위해 예약할 데이터의 일부
    subset=None, #훈련 또는 검중 중 하나
    interpolation="bilinear", #문자열, 이미지 크기를 조정할 때 사용되는 방법, 기본값 : bilinear 그 외 nearest, bicubic, area, lanczos3, lanczos5, gaussian 를 지원
    follow_links=False, #심볼릭 링크가 가리키는 하위 디렉토리를 방문할지 여부, 기본값 : false
    crop_to_aspect_ratio=False, #True의 경우 가로 세로 비율 왜곡 없이 이미지를 조정
    **kwargs #레거시 키워드 인수
)

In [3]:
cnn = tf.keras.Sequential()
cnn.add(tf.keras.layers.Conv2D(filters=64, padding='same', strides=2, kernel_size=3, activation='relu', input_shape=(224,224,3)))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

cnn.add(tf.keras.layers.Conv2D(filters=32, padding='same', strides=2, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

cnn.add(tf.keras.layers.Conv2D(filters=32, padding='same', strides=2, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2))

cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(4, activation='softmax'))

tf.keras.layers.Conv2DTranspose(
    filters, kernel_size, strides=(1, 1), padding='valid', output_padding=None,
    data_format=None, dilation_rate=(1, 1), activation=None, use_bias=True,
    kernel_initializer='glorot_uniform', bias_initializer='zeros',
    kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None,
    kernel_constraint=None, bias_constraint=None, **kwargs
)

filters - [Integer] convolution filter의 수이다. 몇개을 쓸 것인가를 나타낸다. 즉, ootput feature의 채널의 나타낸다.,
kernel_size - [integer, tuple, list] convolution filter를 3x3으로 할지 5x5로 할지 정하는 것이다. 즉 3만 써도 되고 튜플 형태로 (3,3)을 써도 되고 [3, 3]으로 써도 된다.
strides - [kernel_size와 동일] 몇칸을 움직일 것인가
paddin - valid를 써도 되고 same을 써도된다.
valid는 padding을 안하는 것이다. same은 strides가 1인경우를 기준으로 했는데 입력과 출력의 size 가 같아지게 만들어서 size가 줄어들지 않게 하여 여러번 반복이 가능하게 만든다.
data_format - channels_last가 디폴트 값이고 (batch, height, width, channels) 순서로 써야한다. channels_first의 경우는 (batch, channels, height, width) 순서로 반드시 써야한다. 즉, 입력 이미지나 인풋 피춰 맵이 들어갈 때도 마찬가지로 이렇게 4차원 tensor를 적어줘야한다.
activation - activation funtion을 넣는 것이다.
use_bias - bias를 쓸것인지.
kernel_initializer, bias_initializer - convolution filter와 bias를 initializer할 때 어떻게 해줄 것인지 정하는 것이다.
kernel_regularizer, bias_regularizer -regularizer 관련해서 기술해 주는 것이다.
우리가 keras.layers같은 하이레벨의 API를 쓸 경우에 실제로 convolution filter는 아래의 순서로 가진다.
kernel dimension : {height, width, in_channel, out_channel}
height, width, in_channel 은 convolution filter의 형태에 관한 것이고 out_channel은 convolution filter의 갯수에 관한 것이다.

In [4]:
cnn.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [5]:
history = cnn.fit(train_datagen, epochs=20, validation_data=test_datagen)

Epoch 1/20


InvalidArgumentError: Graph execution error:

Detected at node 'categorical_crossentropy/softmax_cross_entropy_with_logits' defined at (most recent call last):
    File "/usr/local/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/usr/local/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/jwjang/tf28/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/jwjang/tf28/lib/python3.9/site-packages/traitlets/config/application.py", line 978, in launch_instance
      app.start()
    File "/home/jwjang/tf28/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/home/jwjang/tf28/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "/usr/local/lib/python3.9/asyncio/base_events.py", line 596, in r

: 